In [1]:
import os
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset, DatasetDict
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
# ====== Step 1: Path setup ======
DATA_PATH = r"C:\Users\atharva\CDAC\Project Ner\ner_crime_training_300_sentences.txt"
MODEL_NAME = "microsoft/deberta-v3-base"
OUTPUT_DIR = "./ner_deberta_model"


In [5]:
# ====== Step 2: Load tokenizer ======
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


C:\New folder\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\New folder\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
# ====== Step 3: Read and structure CoNLL data ======
def read_conll(filepath):
    tokens, labels, all_data = [], [], []
    with open(filepath, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens and labels and len(tokens) == len(labels):
                    all_data.append({"tokens": tokens, "ner_tags": labels})
                tokens, labels = [], []
                continue
            splits = line.split()
            if len(splits) >= 2:
                tokens.append(splits[0])
                labels.append(splits[-1])
    if tokens and labels and len(tokens) == len(labels):
        all_data.append({"tokens": tokens, "ner_tags": labels})
    return all_data

all_data = read_conll(DATA_PATH)


In [7]:
# ====== Step 4: Label encoding (FIXED) ======
unique_labels = sorted(set(label for row in all_data for label in row["ner_tags"]))
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {i: label for label, i in label_to_id.items()}

# ✅ Convert all ner_tags to integer IDs
for row in all_data:
    encoded_tags = []
    for label in row["ner_tags"]:
        if label in label_to_id:
            encoded_tags.append(label_to_id[label])
        else:
            raise ValueError(f"Unknown label found: {label}")
    row["ner_tags"] = encoded_tags


In [8]:
# ====== Step 5: Convert to HuggingFace Dataset ======
dataset = Dataset.from_list(all_data)
if len(all_data) >= 10:
    dataset = dataset.train_test_split(test_size=0.1)
    datasets = DatasetDict(train=dataset["train"], test=dataset["test"])
else:
    datasets = DatasetDict(train=dataset, test=dataset)


In [13]:
# ====== Step 6: Tokenize and align labels ======
def tokenize_and_align_labels(examples):
    # Add max_length to suppress truncation warning
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        max_length=512,
        is_split_into_words=True
    )
    
    labels = []
    for i, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=i) for i in range(len(examples["tokens"]))):
        label_ids = []
        prev_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != prev_word_idx:
                try:
                    label_ids.append(examples["ner_tags"][i][word_idx])
                except Exception as e:
                    print("⚠️ Error aligning labels:", e)
                    print("Tokens:", examples["tokens"][i])
                    print("NER tags:", examples["ner_tags"][i])
                    raise
            else:
                label_ids.append(-100)
            prev_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 🧪 Safe check before mapping
for row in all_data:
    for label in row["ner_tags"]:
        assert isinstance(label, int), f"❌ Found non-integer label: {label}"

# 🔁 Apply to dataset
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=datasets["train"].column_names
)


Map:   0%|          | 0/270 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [15]:
# ====== Step 7: Load model ======
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(unique_labels),
    id2label=id_to_label,
    label2id=label_to_id
)


Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# ====== Step 8: Evaluation metrics ======
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    true_labels = []

    for pred, label in zip(predictions, labels):
        temp_pred, temp_label = [], []
        for p_i, l_i in zip(pred, label):
            if l_i != -100:
                temp_pred.append(id_to_label[p_i])
                temp_label.append(id_to_label[l_i])
        true_predictions.append(temp_pred)
        true_labels.append(temp_label)

    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions)
    }


In [19]:
# ====== Step 9: Training arguments ======
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none"
)

data_collator = DataCollatorForTokenClassification(tokenizer)


In [21]:
# ====== Step 10: Trainer setup ======
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# ====== Step 11: Train and save ======
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Training complete. Model saved to:", OUTPUT_DIR)


C:\New folder\Lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.617700,0.081798,0.989691,0.956140,0.956140,0.956140
2,0.052700,0.008653,1.000000,1.000000,1.000000,1.000000
3,0.016200,0.004059,1.000000,1.000000,1.000000,1.000000
4,0.011200,0.003086,1.000000,1.000000,1.000000,1.000000


✅ Training complete. Model saved to: ./ner_deberta_model
